# Chapter 7
# Backtesting and Live Trading

After you are satisfied with the model performance, you can take the signals generated by them to trade and analyse the returns. Not only returns, but you should also analyse the risk associated with generating the returns.

**Why are we backtesting the model?** The majority of the traders in the market lose money. They lose money not because they lack an understanding of the market. But simply because their trading decisions are not based on sound research and tested trading methods.

They make decisions based on emotions, suggestions from friends, and take excessive risks in the hope to get rich quickly. If they remove emotions and instincts from the trading and backtest the ideas before trading, then the chance to trade profitability in the market is increased.

In *backtesting*, you are testing a trading hypothesis / strategy on the historical data.

In the previous chapters, you took the 15-minute, 30-minute, and 75-minute prior percentage change as your features and expected that these features will help you predict the future returns. **This is your hyphotesis.**

**How would you test this hypothesis? How would you know whether the strategy will work in the market or not?** By using historical data, you can backtest and see whether your hypothesis is true or not. It helps assess the feasibility of a trading strategy by discovering how it performs on the historical data.

In [6]:
# For data manipulation
import numpy as np
import pandas as pd

# For plotting
import matplotlib.pyplot as plt


# Read the data
# We are going to read the close price
strategy_data = pd.DataFrame()
strategy_data['close'] = pd.read_csv('/Users/nacho/Documents/GitHub/python_course/books/Machine-Learning-Trading/assets/machine-learning-in-trading-main-main/data_modules/JPM_2017_2019.csv',
    index_col=0)['close']

# Read the predicted signals
strategy_data['predicted_signal'] = pd.read_csv('/Users/nacho/Documents/GitHub/python_course/books/Machine-Learning-Trading/assets/machine-learning-in-trading-main-main/data_modules/JPM_predicted_2017_2019.csv',
    index_col=0)

strategy_data.index = pd.to_datetime(strategy_data.index)
strategy_data = strategy_data.dropna()

# Preview the strategy data
strategy_data.head()

,close,predicted_signal
2019-05-28 12:00:00+00:00,109.29,0.0
2019-05-28 12:15:00+00:00,109.37,0.0
2019-05-28 12:30:00+00:00,109.33,1.0
2019-05-28 12:45:00+00:00,109.37,0.0
2019-05-28 13:00:00+00:00,109.38,0.0


The first thing you want to know from your strategy is what the returns are. Only then will you think if the trade made sense. So you will first calculate the strategy returns, as shown below.

In [7]:
# Calculate the percentage change
strategy_data['pct_change'] = strategy_data['close'].pct_change()

# Calculate the strategy returns
strategy_data['strategy_returns'] = strategy_data['predicted_signal'].shift(1) * \
    strategy_data['pct_change']

# Drop the missing values
strategy_data.dropna(inplace=True)

strategy_data[['strategy_returns']].head()

,strategy_returns
2019-05-28 12:15:00+00:00,0.000000
2019-05-28 12:30:00+00:00,-0.000000
2019-05-28 12:45:00+00:00,0.000366
2019-05-28 13:00:00+00:00,0.000000
2019-05-28 13:15:00+00:00,-0.000000


**Trade Logs** We'll create a function `get_trades`. The function will take the dataframe with the signals and close price as input along with the cloumn names for signals and close price.

It will return a dataframe containing the details of each trade that was taken during backtest. The following details are included for each trade:

+ Trade direction (position)
+ Entry time
+ Entry price
+ Exit time
+ Exit price
+ Profit or loss
  

In [9]:
def get_trades(data, close_column, signal_column):
    # Create a dataframe to store value of each trade
    trades = pd.DataFrame()

    # Set current position as and entry time as empty
    current_position = 0
    entry_time = ''

    # Loop through each trade in dataframe
    for i in data.index:
        # Each row in the dataframe will be treated as a new position
        new_position = data.loc[i, signal_column]

        # If the position changes, record the entry and exit
        if new_position!= current_position:
            if entry_time != '':
                entry_price = data.loc[entry_time, close_column]
                exit_time = i 
                exit_price = data.loc[exit_time, close_column]
                trade_details = pd.DataFrame(
                    [(current_position, entry_time, entry_price, exit_time, exit_price)])
                
                # trades = trades.append(trade_details, ignore_index=True)

                # df = pd.DataFrame(df).append(new_row, ignore_index=True)
                # df = pd.concat([df, pd.DataFrame([new_row])], ignore_index=True)

                trades = trades.concat([trades, pd.DataFrame([trade_details])], ignore_index=True)

                entry_time = ''

            if new_position != 0:
                entry_time = i
            current_position = new_position

    # Add the trade details to the 'trades' dataframe
    trades.columns = ['Position', 'Entry Time', 'Entry Price', 'Exit Time', 'Exit Price']
    trades['PnL'] = (trades['Exit Price'] - trades['Entry Price']) * trades['Position']

    return trades

# Get trade details for hit ratio strategy
trade_log = get_trades(strategy_data, 'close', 'predicted_signal')

trade_log,iloc[0].T


AttributeError: 'DataFrame' object has no attribute 'concat'